In [1]:
import numpy as np
import pandas as pd
import xlsxwriter

import gurobipy as gp
from gurobipy import GRB


np.random.seed(0)

In [2]:
####################### INDEX ################################

j = 3  #Customer sub index
m = 1  #Manufacturer sub index
g = 1  #GMSD index
s = 1  #State sub index
d = 38  #District sub index
i = 151 #Clinic sub index
t = 12  #Time sub index

clinic_breakpoints = [10,16,28,40,59,76,92,103,123,151,178,194,205,213,226,249,256,266,274,290,312,322,340,361,376,413,432,451,462,483,504,511,517,535,555,568,585,606]   #CLinic breakpoints for each districts
clinic_breakpoints = clinic_breakpoints[0:d]
i = clinic_breakpoints[d-1]

customers = list(range(1,j+1))
manufacturers = list(range(1,m+1))
gmsd = list(range(1,g+1))
svs = list(range(1,s+1))
dvs = list(range(1,d+1))
clinics = list(range(1,i+1))
time = list(range(1,t+1))

Lm = 0
Lg = 0
Ls = 0
Ld = 0
Li = 0
X = list(range(0,t-Li+1))

Ig_0 = 0
Is_0 = 0
Id_0 = 0
Ii_0 = 0

top_dvs = 5

In [3]:
########################### PARAMETERS ################################
l = GRB.INFINITY #large number for consistency constraints
fraction_storage = 0.5904 #Fraction of total capacity in cold chain points to be considered for COVID-19 vaccine
fraction_transport = 0.5904 #Fraction of total capacity in vehicles to be considered for COVID-19 vaccine

In [4]:
#Transportation cost
diesel_cost = 14
ftc_fac = 1
booking_cost = {
	"MG" : 40000,
	"GS" : 20000,
	"SD" : 12000*ftc_fac,
	"DI" : 5000*ftc_fac
}

#Distances
Dgm = [[1000]] #From M to G (confirm)
Dsg = [[550]] #From G to S (confirm)

#From S to D
df_Dds = pd.read_csv("Input_data/distances_sd.csv")
Dds = [[0 for D in range(d)] for S in range(s)]
for index in df_Dds.index:
    if (df_Dds['d'][index] > d):
        continue
    Dds[df_Dds['s'][index]-1][df_Dds['d'][index]-1] = df_Dds['Distance'][index]
    


#From D to I
df_Did = pd.read_csv("Input_data/distances_di.csv")
Did = [[0 for I in range(i)] for D in range(d)]
for index in df_Did.index:
    if (df_Did['d'][index] > d or df_Did['i'][index] > i):
        continue
    Did[df_Did['d'][index]-1][df_Did['i'][index]-1] = df_Did['Distance'][index]
    if(df_Did['d'][index]==1):
        if(df_Did['i'][index] not in range(0+1,clinic_breakpoints[df_Did['d'][index]-1]+1)):
            Did[df_Did['d'][index]-1][df_Did['i'][index]-1] = df_Did['Distance'][index]*1   
    else: 
        if(df_Did['i'][index] not in range(clinic_breakpoints[df_Did['d'][index]-2]+1,clinic_breakpoints[df_Did['d'][index]-1]+1)):
            Did[df_Did['d'][index]-1][df_Did['i'][index]-1] = df_Did['Distance'][index]*1

#Capacity of trucks
cap_veh_gm = round(fraction_transport*2932075) #Refrigerated van
cap_veh_sg = round(fraction_transport*2932075) #Refrigerated van
cap_veh_ds = round(fraction_transport*290880) #Insulated van
cap_veh_id = round(fraction_transport*290880) #Insulated van


#Final transportation costs
Kgmt = np.array([[[Dgm[M][G]*diesel_cost+booking_cost["MG"] for G in range(0,g)] for M in range(0,m)] for T in range(0,t)])
Ksgt = np.array([[[Dsg[G][S]*diesel_cost+booking_cost["GS"] for S in range(0,s)] for G in range(0,g)] for T in range(0,t)])
Kdst = np.array([[[Dds[S][D]*diesel_cost+booking_cost["SD"] for D in range(0,d)] for S in range(0,s)] for T in range(0,t)])
Kidt = np.array([[[Did[D][I]*diesel_cost+booking_cost["DI"] for I in range(0,i)] for D in range(0,d)] for T in range(0,t)])

In [5]:
########################## Finding top dvs from clinics ####################
top_dvs_arr = [[0 for D in range(top_dvs)] for I in clinics]
for I in clinics:
    temp = [(Did[D-1][I-1], D) for D in dvs]
    temp.sort()
    top_dvs_arr[I-1] = [temp[D][1] for D in range(top_dvs)]
    print(I)
    print(top_dvs_arr[I-1])
    print("")


1
[1, 27, 18, 16, 20]

2
[1, 27, 18, 16, 20]

3
[1, 20, 27, 36, 29]

4
[1, 27, 36, 20, 18]

5
[1, 27, 18, 16, 20]

6
[1, 27, 18, 36, 16]

7
[1, 36, 20, 27, 29]

8
[1, 18, 27, 16, 20]

9
[1, 27, 18, 16, 20]

10
[1, 27, 18, 16, 36]

11
[2, 14, 7, 26, 3]

12
[2, 14, 7, 26, 3]

13
[2, 3, 14, 7, 26]

14
[2, 14, 7, 26, 3]

15
[14, 2, 11, 26, 7]

16
[2, 14, 7, 26, 3]

17
[3, 15, 28, 2, 7]

18
[3, 2, 14, 11, 7]

19
[3, 11, 2, 14, 15]

20
[2, 11, 3, 14, 7]

21
[2, 3, 14, 11, 7]

22
[3, 2, 11, 15, 28]

23
[3, 2, 11, 15, 28]

24
[3, 11, 2, 14, 15]

25
[3, 2, 15, 28, 11]

26
[3, 2, 11, 15, 28]

27
[3, 2, 14, 7, 8]

28
[3, 2, 11, 15, 28]

29
[4, 6, 22, 13, 17]

30
[4, 6, 22, 13, 19]

31
[4, 6, 22, 13, 19]

32
[4, 6, 22, 13, 17]

33
[4, 6, 22, 13, 19]

34
[4, 13, 22, 6, 19]

35
[4, 13, 6, 19, 22]

36
[4, 6, 22, 13, 17]

37
[4, 6, 22, 13, 19]

38
[4, 6, 22, 13, 19]

39
[4, 6, 22, 13, 17]

40
[4, 6, 22, 13, 19]

41
[5, 30, 19, 17, 9]

42
[5, 17, 19, 30, 32]

43
[5, 17, 19, 30, 32]

44
[5, 19, 30, 17, 


399
[26, 14, 2, 7, 37]

400
[26, 14, 37, 7, 2]

401
[2, 14, 26, 7, 11]

402
[26, 5, 19, 32, 24]

403
[26, 14, 7, 2, 37]

404
[26, 14, 7, 2, 37]

405
[26, 37, 14, 7, 2]

406
[26, 14, 7, 2, 37]

407
[26, 14, 2, 37, 7]

408
[26, 14, 37, 7, 2]

409
[26, 14, 7, 2, 37]

410
[26, 14, 37, 7, 23]

411
[26, 14, 7, 2, 37]

412
[26, 7, 37, 14, 2]

413
[14, 26, 2, 24, 7]

414
[27, 16, 1, 18, 20]

415
[27, 16, 1, 18, 20]

416
[27, 16, 1, 18, 20]

417
[27, 16, 1, 18, 20]

418
[27, 16, 1, 18, 20]

419
[27, 16, 1, 18, 20]

420
[27, 16, 1, 18, 20]

421
[27, 16, 1, 18, 20]

422
[27, 16, 1, 18, 20]

423
[27, 16, 1, 18, 20]

424
[27, 1, 16, 18, 20]

425
[27, 16, 1, 18, 20]

426
[27, 16, 1, 18, 20]

427
[27, 16, 1, 18, 20]

428
[27, 16, 1, 18, 20]

429
[27, 16, 1, 18, 20]

430
[27, 16, 1, 18, 20]

431
[27, 16, 1, 18, 20]

432
[27, 1, 16, 20, 18]

433
[28, 15, 3, 2, 8]

434
[28, 15, 3, 2, 8]

435
[15, 3, 28, 8, 2]

436
[8, 2, 7, 3, 15]

437
[3, 15, 2, 28, 7]

438
[8, 15, 7, 3, 2]

439
[8, 3, 2, 7, 15]

440


In [6]:
#Shortage costs
SC = 1
Pjt = [[0 for J in range(j)] for T in range(t)]
for T in range(t):
    Pjt[T][0] = 75000*SC       #children
    Pjt[T][1] = 50000*SC       #adults
    Pjt[T][2] = 50000*SC       #elderly


#Clinical cost per unit of vaccine
Vj = 225

np.random.seed(0)
#Inventory holding costs
hgt = [[0.3 for G in range(g)] for T in range(t)]
hst = [[0.3 for S in range(s)] for T in range(t)]
hdt = np.random.normal(0.4,0,d*t).reshape(t,d)
hit = np.random.normal(0.4,0,i*t).reshape(t,i)

oc_fac = 1
#Ordering costs
Cgmt = [[[200000 for G in range(g)] for M in range(m)] for T in range(t)]
Csgt = [[[100000 for S in range(s)] for G in range(g)] for T in range(t)]
Cdst = [[[75000*oc_fac for D in range(d)] for S in range(s)] for T in range(t)]
Cidt = [[[15000*oc_fac for I in range(i)] for D in range(d)] for T in range(t)]

#Demand
wastage_factor = 0.5 #This value will depend on the vaccine, we are talking about.

In [7]:
#Fraction of demand
Fr_d = 1
#Fr_d = 0.5
#Fr_d = 0.75

df_demand = pd.read_csv("Input_data/weekly_demand.csv")
dijt = [[[0 for I in range(1,i+1)] for J in range(j)] for T in range(1,t+1)]
for index in df_demand.index:
    if (df_demand['i'][index] > i):
        break
    if(df_demand['t'][index]>t):
        continue
    dijt[df_demand['t'][index]-1][df_demand['j'][index]-1][df_demand['i'][index]-1] = round(df_demand['demand'][index]*Fr_d)

#Capacity of cold chain points
Bgt = [[round(fraction_storage*24545455) for G in range(g)] for T in range(t)]
Bst = [[round(fraction_storage*6818182) for S in range(s)] for T in range(t)]


df_bdt = pd.read_csv("Input_data/capacity_DVS.csv")
Bdt = [[0 for D in range(d)] for T in range(t)]
for index in df_bdt.index:
    if(df_bdt['d'][index] > d):
        break
    if(df_bdt['t'][index]>t):
        continue
    Bdt[df_bdt['t'][index]-1][df_bdt['d'][index]-1] = fraction_storage*df_bdt['Capacity'][index]


df_bit = pd.read_csv("Input_data/capacity_clinics.csv")
Bit = [[0 for I in range(i)] for T in range(t)]
for index in df_bit.index:
    if(df_bit['i'][index] > i):
        break
    if(df_bit['t'][index]>t):
        continue
    Bit[df_bit['t'][index]-1][df_bit['i'][index]-1] = fraction_storage*df_bit['Capacity'][index]



model = gp.Model('Vaccine_Distribution')

Academic license - for non-commercial use only - expires 2021-06-02
Using license file C:\Users\Ayush\gurobi.lic


In [8]:
#Production Capacity
# Bmt = [[round(350000*d/38) for M in range(m)] for T in range(t)]
Bmt = [[20000000 for M in range(m)] for T in range(t)]

#Average time required to administer the vaccine (minutes)
No = 5

#Number of medical personnel hours available (minutes)
Nit = [[3360 for I in range(i)] for T in range(t)]

#Hiring Cost of nurses
hc = 25000

#Firing Cost of nurses
fc = 10000

#Weekly wages of nurses
wg = 6175

#cap on trucks
cap_gm = 3
cap_sg = 3
cap_ds = 5
cap_id = 5

In [9]:
################### DECISION VARIABLES ##########################

#Inventory
Igt = model.addVars(time,gmsd,vtype=GRB.INTEGER, name="Igt")
# Igt = model.addVars(time,gmsd,vtype=GRB.CONTINUOUS, name="Igt")
Ist = model.addVars(time,svs,vtype=GRB.INTEGER, name="Ist")
# Ist = model.addVars(time,svs,vtype=GRB.CONTINUOUS, name="Ist")
Idt = model.addVars(time,dvs,vtype=GRB.INTEGER, name="Idt")
# Idt = model.addVars(time,dvs,vtype=GRB.CONTINUOUS, name="Idt")
Iit = model.addVars(time,clinics,vtype=GRB.INTEGER, name="Iit")
# Iit = model.addVars(time,clinics,vtype=GRB.CONTINUOUS, name="Iit")

#Quantity
Qgmt = model.addVars(time,manufacturers,gmsd,vtype=GRB.INTEGER, name="Qgmt")
# Qgmt = model.addVars(time,manufacturers,gmsd,vtype=GRB.CONTINUOUS, name="Qgmt")
Qsgt = model.addVars(time,gmsd,svs,vtype=GRB.INTEGER, name="Qsgt")
# Qsgt = model.addVars(time,gmsd,svs,vtype=GRB.CONTINUOUS, name="Qsgt")
Qdst = model.addVars(time,svs,dvs,vtype=GRB.INTEGER, name="Qdst")
# Qrst = model.addVars(time,svs,rvs,vtype=GRB.CONTINUOUS, name="Qrst")
Qidt = model.addVars(time,dvs,clinics,vtype=GRB.INTEGER, name="Qidt")
# Qidt = model.addVars(time,dvs,clinics,vtype=GRB.CONTINUOUS, name="Qidt")

#Shortage and Consumption
Sijt = model.addVars(time,customers,clinics,vtype=GRB.INTEGER, name="Sijt")
# Sijt = model.addVars(time,customers,clinics,vtype=GRB.CONTINUOUS, name="Sijt")
Wijt = model.addVars(time,customers,clinics,vtype=GRB.INTEGER, name="Wijt")
# Wijt = model.addVars(time,customers,clinics,vtype=GRB.CONTINUOUS, name="Wijt")

#Assignment Variables
Xgmt = model.addVars(time,manufacturers,gmsd,vtype=GRB.BINARY, name="Xgmt")
Xsgt = model.addVars(time,gmsd,svs,vtype=GRB.BINARY, name="Xsgt")
Xdst = model.addVars(time,svs,dvs,vtype=GRB.BINARY, name="Xdst")
Xidt = model.addVars(time,dvs,clinics,vtype=GRB.BINARY, name="Xidt")

#Number of trucks
Ngmt = model.addVars(time,manufacturers,gmsd,vtype=GRB.INTEGER, name="Ngmt")
Nsgt = model.addVars(time,gmsd,svs,vtype=GRB.INTEGER, name="Nsgt")
Ndst = model.addVars(time,svs,dvs,vtype=GRB.INTEGER, name="Ndst")
Nidt = model.addVars(time,dvs,clinics,vtype=GRB.INTEGER, name="Nidt")

#Nurses
N_nurses_it = model.addVars(time,clinics,vtype=GRB.INTEGER, name = "N_nurses_it")
H_nurses_it = model.addVars(time,clinics,vtype=GRB.INTEGER, name = "H_nurses_it")
F_nurses_it = model.addVars(time,clinics,vtype=GRB.INTEGER, name = "F_nurses_it")

In [10]:
############################# OBJECTIVE FUNCTION ###########################
transport_part = gp.quicksum(Kgmt[T-1][M-1][G-1]*Ngmt[T,M,G] for G in gmsd for M in manufacturers for T in time)
transport_part += gp.quicksum(Ksgt[T-1][G-1][S-1]*Nsgt[T,G,S] for S in svs for G in gmsd for T in time)
transport_part += gp.quicksum(Kdst[T-1][S-1][D-1]*Ndst[T,S,D] for D in dvs for S in svs for T in time)
transport_part += gp.quicksum(Kidt[T-1][D-1][I-1]*Nidt[T,D,I] for I in clinics for D in dvs for T in time)

inventory_part = gp.quicksum(hgt[T-1][G-1]*Igt[T,G] for G in gmsd for T in time)
inventory_part += gp.quicksum(hst[T-1][S-1]*Ist[T,S] for S in svs for T in time)
inventory_part += gp.quicksum(hdt[T-1][D-1]*Idt[T,D] for D in dvs for T in time)
inventory_part += gp.quicksum(hit[T-1][I-1]*Iit[T,I] for I in clinics for T in time)

shortage_part = gp.quicksum(Pjt[T-1][J-1]*Sijt[T,J,I] for J in customers for I in clinics for T in time)

consumption_part = gp.quicksum(Vj*Wijt[T,J,I] for J in customers for I in clinics for T in time) #Do we even need this?

ordering_part = gp.quicksum(Cgmt[T-1][M-1][G-1]*Xgmt[T,M,G] for G in gmsd for M in manufacturers for T in time)
ordering_part += gp.quicksum(Csgt[T-1][G-1][S-1]*Xsgt[T,G,S] for S in svs for G in gmsd for T in time)
ordering_part += gp.quicksum(Cdst[T-1][S-1][D-1]*Xdst[T,S,D] for D in dvs for S in svs for T in time)
ordering_part += gp.quicksum(Cidt[T-1][D-1][I-1]*Xidt[T,D,I] for I in clinics for D in dvs for T in time)

nurses_part = gp.quicksum(wg*N_nurses_it[T,I] for I in clinics for T in time)
nurses_part += gp.quicksum(hc*H_nurses_it[T,I] for I in clinics for T in time)
nurses_part += gp.quicksum(fc*F_nurses_it[T,I] for I in clinics for T in time)

model.setObjective(transport_part+inventory_part+shortage_part+consumption_part+ordering_part+nurses_part,GRB.MINIMIZE)


In [11]:
###################################### CONSTRAINTS ################################
#Pre-Processing
# pre_proc_1 = model.addConstrs(((Qidt[T,D,I] if D not in top_dvs_arr[I-1] else 0) == 0 for T in time for D in dvs for I in clinics),name = "pre_proc_1")
# pre_proc_2 = model.addConstrs(((Xidt[T,D,I] if D not in top_dvs_arr[I-1] else 0) == 0 for T in time for D in dvs for I in clinics),name = "pre_proc_2")
# pre_proc_3 = model.addConstrs(((Nidt[T,D,I] if D not in top_dvs_arr[I-1] else 0) == 0 for T in time for D in dvs for I in clinics),name = "pre_proc_3")


#Inventory Balance
gmsd_inventory = model.addConstrs(((Igt[T-1,G] if T>1 else Ig_0) + gp.quicksum((Qgmt[T-Lm,M,G] if T>Lm else 0) for M in manufacturers) 
                                    - Igt[T,G] == gp.quicksum(Qsgt[T,G,S] for S in svs) for G in gmsd for T in time),
                                     name="gmsd_inventory")
svs_inventory = model.addConstrs(((Ist[T-1,S] if T>1 else Is_0) + gp.quicksum((Qsgt[T-Lg,G,S] if T>Lg else 0) for G in gmsd) 
                                    - Ist[T,S] == gp.quicksum(Qdst[T,S,D] for D in dvs) for S in svs for T in time),
                                     name="svs_inventory")
dvs_inventory = model.addConstrs(((Idt[T-1,D] if T>1 else Id_0) + gp.quicksum((Qdst[T-Ls,S,D] if T>Ls else 0) for S in svs) 
                                    - Idt[T,D] == gp.quicksum(Qidt[T,D,I] for I in clinics) for D in dvs for T in time),
                                     name="dvs_inventory")
clinic_inventory = model.addConstrs(((Iit[T-1,I] if T>1 else Ii_0) + gp.quicksum((Qidt[T-Ld,D,I] if T>Ld else 0) for D in dvs) 
                                    - Iit[T,I] == gp.quicksum(Wijt[T,J,I] for J in customers) for I in clinics for T in time),
                                     name="clinic_inventory")

#Consumption by demand
consumption_demand = model.addConstrs((Wijt[T,J,I] <= dijt[T-1][J-1][I-1] for I in clinics for J in customers for T in time),
                                    name = "consumption_demand")

# Administration constraint
if(Li>0):
    administer_const = model.addConstrs((gp.quicksum(Wijt[T,J,I] for J in customers) <= (Iit[T-Li,I] if T>Li else Ii_0)
                                         for I in clinics for T in time),name = "administer_const")

#Consumption Balance
consumption_balance = model.addConstrs((Wijt[T,J,I] + Sijt[T,J,I] == dijt[T-1][J-1][I-1] for I in clinics for J in customers for T in time),
                                    name = "consumption_balance")

#Inventory Capacity constraints
gmsd_cap = model.addConstrs((Igt[T,G]<=Bgt[T-1][G-1] for G in gmsd for T in time),name = "gmsd_cap")
svs_cap = model.addConstrs((Ist[T,S]<=Bst[T-1][S-1] for S in svs for T in time),name = "svs_cap")
dvs_cap = model.addConstrs((Idt[T,D]<=Bdt[T-1][D-1] for D in dvs for T in time),name = "dvs_cap")
clinic_cap = model.addConstrs((Iit[T,I]<=Bit[T-1][I-1] for I in clinics for T in time),name = "clinic_cap")

#Production capacity constraints
production_cap = model.addConstrs((gp.quicksum(Qgmt[T,M,G] for G in gmsd)<= Bmt[T-1][M-1] for M in manufacturers for T in time)
                                ,name = "production_cap")

#Facility selection constraints
fac_sg = model.addConstrs((gp.quicksum(Xsgt[T,G,S] for G in gmsd)<=1 for S in svs for T in time),name = "fac_sg")
fac_ds = model.addConstrs((gp.quicksum(Xdst[T,S,D] for S in svs)<=1 for D in dvs for T in time),name = "fac_ds")
fac_id = model.addConstrs((gp.quicksum(Xidt[T,D,I] for D in dvs)<=1 for I in clinics for T in time),name = "fac_id")

#Constraints for consistency of q and X
# cons_1 = model.addConstrs((Xgmt[T,M,G]<=l*Qgmt[T,M,G] for G in gmsd for M in manufacturers for T in time),name = "cons_1")
cons_2 = model.addConstrs((Qgmt[T,M,G]<=cap_veh_gm*cap_gm*Xgmt[T,M,G] for G in gmsd for M in manufacturers for T in time),name = "cons_2")
# cons_3 = model.addConstrs((Xsgt[T,G,S]<=l*Qsgt[T,G,S] for S in svs for G in gmsd for T in time),name = "cons_3")
cons_4 = model.addConstrs((Qsgt[T,G,S]<=cap_veh_sg*cap_sg*Xsgt[T,G,S] for S in svs for G in gmsd for T in time),name = "cons_4")
# cons_5 = model.addConstrs((Xrst[T,S,R]<=l*Qrst[T,S,R] for R in rvs for S in svs for T in time),name = "cons_5")
cons_6 = model.addConstrs((Qdst[T,S,D]<=cap_veh_ds*cap_ds*Xdst[T,S,D] for D in dvs for S in svs for T in time),name = "cons_6")
# cons_9 = model.addConstrs((Xidt[T,D,I]<=l*Qidt[T,D,I] for I in clinics for D in dvs for T in time),name = "cons_9")
cons_10 = model.addConstrs((Qidt[T,D,I]<=cap_veh_id*cap_id*Xidt[T,D,I] for I in clinics for D in dvs for T in time),name = "cons_10")

# cons_1 = model.addConstrs((Xgmt[T,M,G]<=l*Qgmt[T,M,G] for G in gmsd for M in manufacturers for T in time),name = "cons_1")
# cons_2 = model.addConstrs((Qgmt[T,M,G]<=l*Xgmt[T,M,G] for G in gmsd for M in manufacturers for T in time),name = "cons_2")
# cons_3 = model.addConstrs((Xsgt[T,G,S]<=l*Qsgt[T,G,S] for S in svs for G in gmsd for T in time),name = "cons_3")
# cons_4 = model.addConstrs((Qsgt[T,G,S]<=l*Xsgt[T,G,S] for S in svs for G in gmsd for T in time),name = "cons_4")
# cons_5 = model.addConstrs((Xrst[T,S,R]<=l*Qrst[T,S,R] for R in rvs for S in svs for T in time),name = "cons_5")
# cons_6 = model.addConstrs((Qrst[T,S,R]<=l*Xrst[T,S,R] for R in rvs for S in svs for T in time),name = "cons_6")
# cons_7 = model.addConstrs((Xdrt[T,R,D]<=l*Qdrt[T,R,D] for D in dvs for R in rvs for T in time),name = "cons_7")
# cons_8 = model.addConstrs((Qdrt[T,R,D]<=l*Xdrt[T,R,D] for D in dvs for R in rvs for T in time),name = "cons_8")
# cons_9 = model.addConstrs((Xidt[T,D,I]<=l*Qidt[T,D,I] for I in clinics for D in dvs for T in time),name = "cons_9")
# cons_10 = model.addConstrs((Qidt[T,D,I]<=l*Xidt[T,D,I] for I in clinics for D in dvs for T in time),name = "cons_10")

#Number of trucks constraints
num_trucks_1 = model.addConstrs((Qgmt[T,M,G]/cap_veh_gm<=Ngmt[T,M,G] for G in gmsd for M in manufacturers for T in time),name = "num_trucks_1")
# num_trucks_2 = model.addConstrs((Ngmt[T,M,G]-Qgmt[T,M,G]/cap_veh_gm<=((cap_veh_gm-1)/cap_veh_gm) for G in gmsd for M in manufacturers for T in time),name = "num_trucks_2")
num_trucks_3 = model.addConstrs((Qsgt[T,G,S]/cap_veh_sg<=Nsgt[T,G,S] for S in svs for G in gmsd for T in time),name = "num_trucks_3")
# num_trucks_4 = model.addConstrs((Nsgt[T,G,S]-Qsgt[T,G,S]/cap_veh_sg<=((cap_veh_sg-1)/cap_veh_sg) for S in svs for G in gmsd for T in time),name = "num_trucks_4")
num_trucks_5 = model.addConstrs((Qdst[T,S,D]/cap_veh_ds<=Ndst[T,S,D] for D in dvs for S in svs for T in time),name = "num_trucks_5")
# num_trucks_6 = model.addConstrs((Nrst[T,S,R]-Qrst[T,S,R]/cap_veh_rs<=((cap_veh_rs-1)/cap_veh_rs) for R in rvs for S in svs for T in time),name = "num_trucks_6")
num_trucks_9 = model.addConstrs((Qidt[T,D,I]/cap_veh_id<=Nidt[T,D,I] for I in clinics for D in dvs for T in time),name = "num_trucks_9")
# num_trucks_10 = model.addConstrs((Nidt[T,D,I]-Qidt[T,D,I]/cap_veh_id<=((cap_veh_id-1)/cap_veh_id) for I in clinics for D in dvs for T in time),name = "num_trucks_10")

#Medical personnel availability constraints
med_constraint = model.addConstrs((gp.quicksum(No*Wijt[T,J,I] for J in customers)<=Nit[T-1][I-1]*N_nurses_it[T,I] for I in clinics for T in time),name = "med_constraint")

#Nurses Balance Constraint
nurses_constraint = model.addConstrs((N_nurses_it[T,I] == (N_nurses_it[T-1,I] if T>1 else 0)+ H_nurses_it[T,I] - F_nurses_it[T,I] for I in clinics for T in time),name = "nurses_constraint")

#Cap on trucks constraint
# trucks_constraint_gm = model.addConstrs((Ngmt[T,M,G]<=cap_gm for G in gmsd for M in manufacturers for T in time),name = "trucks_constraint_gm")
# trucks_constraint_sg = model.addConstrs((Nsgt[T,G,S]<=cap_sg for S in svs for G in gmsd for T in time),name = "trucks_constraint_sg")
# trucks_constraint_rs = model.addConstrs((Nrst[T,S,R]<=cap_rs for R in rvs for S in svs for T in time),name = "trucks_constraint_rs")
# trucks_constraint_dr = model.addConstrs((Ndrt[T,R,D]<=cap_dr for D in dvs for R in rvs for T in time),name = "trucks_constraint_dr")
# trucks_constraint_id = model.addConstrs((Nidt[T,D,I]<=cap_id for I in clinics for D in dvs for T in time),name = "trucks_constraint_id")



In [12]:
################################# Solving the problem ##########################
# model.setParam('Method',3)
# model.setParam('Heuristics',1)
model.setParam('MIPGap',1e-6)
model.setParam("TimeLimit", 3600.0)
# model.setParam('MIPFocus',2)
model.optimize()

names = []
sol = []
for v in model.getVars():
    #print(v.varName,"=", v.x)
    names.append(v.varName)
    sol.append(v.x)
print("Done")

Changed value of parameter MIPGap to 1e-06
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 635064 rows, 903648 columns and 2105144 nonzeros
Model fingerprint: 0x8aadacb0
Variable types: 0 continuous, 903648 integer (276816 binary)
Coefficient statistics:
  Matrix range     [6e-07, 5e+06]
  Objective range  [3e-01, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+07]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 1.816934e+12
Presolve removed 52471 rows and 37573 columns (presolve time = 5s) ...
Presolve removed 52471 rows and 37573 columns
Presolve time: 9.56s
Presolved: 582593 rows, 866075 columns, 2007829 nonzeros
Varia

  1199  1239 7.1124e+09  307 26189 7.1576e+09 7.1101e+09  0.66%   291 2930s
  1239  1283 7.1125e+09  318 26084 7.1576e+09 7.1101e+09  0.66%   291 2978s
  1283  1327 7.1125e+09  329 26112 7.1576e+09 7.1101e+09  0.66%   288 3026s
  1327  1369 7.1125e+09  340 25805 7.1576e+09 7.1101e+09  0.66%   285 3073s
  1369  1405 7.1126e+09  351 25593 7.1576e+09 7.1101e+09  0.66%   283 3119s
  1405  1439 7.1126e+09  362 25452 7.1576e+09 7.1101e+09  0.66%   281 3186s
  1439  1481 7.1126e+09  372 25245 7.1576e+09 7.1101e+09  0.66%   281 3244s
  1481  1519 7.1126e+09  382 25046 7.1576e+09 7.1101e+09  0.66%   278 3293s
  1519  1550 7.1127e+09  390 24913 7.1576e+09 7.1101e+09  0.66%   276 3343s
H 1550  1573                    7.157570e+09 7.1101e+09  0.66%   275 3399s
  1573  1606 7.1128e+09  404 24695 7.1576e+09 7.1101e+09  0.66%   276 3455s
H 1589  1606                    7.157568e+09 7.1101e+09  0.66%   275 3455s
  1606  1632 7.1128e+09  416 24591 7.1576e+09 7.1101e+09  0.66%   276 3518s
  1632  1658 7

In [14]:
# ###########################Code for full excel sheet results generation##########################
workbook = xlsxwriter.Workbook("pre"+str(ftc_fac)+' -- '+str(oc_fac)+'districts-'+str(d)+' manuf-'+ str(Bmt[0][0])+'-full-sheet-inventory-ini-'+'.xlsx' )
worksheet = workbook.add_worksheet()
merge_format = workbook.add_format({
    'bold': 1,
    'border': 1, 
    'align': 'center',
    'valign': 'vcenter'})

worksheet.merge_range('A1:B1', 'Inventory', merge_format)
worksheet.merge_range('C1:D1', 'Consumption', merge_format)
worksheet.merge_range('E1:F1', 'Shortage', merge_format)
worksheet.merge_range('G1:H1', 'Delivery from M to G', merge_format)
worksheet.merge_range('I1:J1', 'X from M to G', merge_format)
worksheet.merge_range('K1:L1', 'No of Trucks from M to G', merge_format)
worksheet.merge_range('M1:N1', 'Distance from M to G', merge_format)
worksheet.merge_range('O1:P1', 'Delivery from G to S', merge_format)
worksheet.merge_range('Q1:R1', 'X from G to S', merge_format)
worksheet.merge_range('S1:T1', 'No of Trucks from G to S', merge_format)
worksheet.merge_range('U1:V1', 'Distance from G to S', merge_format)
worksheet.merge_range('W1:X1', 'Delivery from S to D', merge_format)
worksheet.merge_range('Y1:Z1', 'X from S to D', merge_format)
worksheet.merge_range('AA1:AB1', 'No of Trucks from S to D', merge_format)
worksheet.merge_range('AC1:AD1', 'Distance from S to D', merge_format)
worksheet.merge_range('AE1:AF1', 'Delivery from D to I', merge_format)
worksheet.merge_range('AG1:AH1', 'X from D to I', merge_format)
worksheet.merge_range('AI1:AJ1', 'No of Trucks from D to I', merge_format)
worksheet.merge_range('AK1:AL1', 'Distance from D to I', merge_format)


#Inventory
row = 1
for T in time:
    for G in gmsd:
        worksheet.write(row,0,Igt[T,G].varName)
        worksheet.write(row,1,round(Igt[T,G].x))
        row += 1

for T in time:
    for S in svs:
        worksheet.write(row,0,Ist[T,S].varName)
        worksheet.write(row,1,round(Ist[T,S].x))
        row += 1
        
for T in time:
    for D in dvs:
        worksheet.write(row,0,Idt[T,D].varName)
        worksheet.write(row,1,round(Idt[T,D].x))
        row += 1

for T in time:
    for I in clinics:
        worksheet.write(row,0,Iit[T,I].varName)
        worksheet.write(row,1,round(Iit[T,I].x))
        row += 1

#Qgmt, Xgmt, Ngmt
row = 1
for T in time:
    for M in manufacturers:
        for G in gmsd:
            worksheet.write(row,6,Qgmt[T,M,G].varName)
            worksheet.write(row,7,round(Qgmt[T,M,G].x))
            worksheet.write(row,8,Xgmt[T,M,G].varName)
            worksheet.write(row,9,round(Xgmt[T,M,G].x))
            worksheet.write(row,10,Ngmt[T,M,G].varName)
            worksheet.write(row,11,round(Ngmt[T,M,G].x))
            row += 1
#Dgm
row = 1
for M in manufacturers:
    for G in gmsd:
        variable = "Dgm["+str(M)+","+str(G)+"]"
        worksheet.write(row,12,variable)
        worksheet.write(row,13,Dgm[M-1][G-1])
        row += 1
        

#Qsgt,Xsgt,Nsgt
row = 1
for T in time:
    for G in gmsd:
        for S in svs:
            worksheet.write(row,14,Qsgt[T,G,S].varName)
            worksheet.write(row,15,round(Qsgt[T,G,S].x))
            worksheet.write(row,16,Xsgt[T,G,S].varName)
            worksheet.write(row,17,round(Xsgt[T,G,S].x))
            worksheet.write(row,18,Nsgt[T,G,S].varName)
            worksheet.write(row,19,round(Nsgt[T,G,S].x))
            row += 1
#Dsg
row = 1
for G in gmsd:
    for S in svs:
        variable = "Dsg["+str(G)+","+str(S)+"]"
        worksheet.write(row,20,variable)
        worksheet.write(row,21,Dsg[G-1][S-1])
        row += 1

#Qrst,Xrst,Nrst
row = 1
for T in time:
    for S in svs:
        for D in dvs:
            worksheet.write(row,22,Qdst[T,S,D].varName)
            worksheet.write(row,23,round(Qdst[T,S,D].x))
            worksheet.write(row,24,Xdst[T,S,D].varName)
            worksheet.write(row,25,round(Xdst[T,S,D].x))
            worksheet.write(row,26,Ndst[T,S,D].varName)
            worksheet.write(row,27,round(Ndst[T,S,D].x))
            row += 1
#Dds
row = 1
for S in svs:
    for D in dvs:
        variable = "Dds["+str(S)+","+str(D)+"]"
        worksheet.write(row,28,variable)
        worksheet.write(row,29,Dds[S-1][D-1])
        row += 1

        
#Qidt,Xidt,Nidt
row = 1
for T in time:
    for D in dvs:
        for I in clinics:
            worksheet.write(row,30,Qidt[T,D,I].varName)
            worksheet.write(row,31,round(Qidt[T,D,I].x))
            worksheet.write(row,32,Xidt[T,D,I].varName)
            worksheet.write(row,33,round(Xidt[T,D,I].x))
            worksheet.write(row,34,Nidt[T,D,I].varName)
            worksheet.write(row,35,round(Nidt[T,D,I].x))
            row += 1
#Did
row = 1
for D in dvs:
    for I in clinics:
        variable = "Did["+str(D)+","+str(I)+"]"
        worksheet.write(row,36,variable)
        worksheet.write(row,37,Did[D-1][I-1])
        row += 1

#Shortage
row = 1
for T in time:
    for J in customers:
        for I in clinics:
            worksheet.write(row,4,Sijt[T,J,I].varName)
            worksheet.write(row,5,round(Sijt[T,J,I].x))
            row += 1


#Consumption
row = 1
for T in time:
    for J in customers:
        for I in clinics:
            worksheet.write(row,2,Wijt[T,J,I].varName)
            worksheet.write(row,3,round(Wijt[T,J,I].x))
            row += 1



workbook.close()

In [ ]:
############################################ Summary #####################################################

############## transport part ###############
no_of_times_M = ""
cost_M = ""
total_cost_M = 0
for M in manufacturers:
    number = 0
    cost = 0
    for G in gmsd:
        for T in time:
            number =  number + Xgmt[T,M,G].x
            cost = cost + Kgmt[T-1][M-1][G-1]*Ngmt[T,M,G].x
    cost = round(cost)
    if(number!=0):
        if(no_of_times_M==""):
            no_of_times_M += (str(M)+"("+str(number)+" times"+")")
            cost_M += (str(M)+"("+str(cost)+" Rs"+")")
        else:
            no_of_times_M += (", "+str(M)+"("+str(number)+" times"+")")
            cost_M += (", "+str(M)+"("+str(cost)+" Rs"+")")
        total_cost_M += cost


no_of_times_G = ""
cost_G = ""
total_cost_G = 0
for G in gmsd:
    number = 0
    cost = 0
    for S in svs:
        for T in time:
            number =  number + Xsgt[T,G,S].x
            cost = cost + Ksgt[T-1][G-1][S-1]*Nsgt[T,G,S].x
    cost = round(cost)
    if(number!=0):
        if(no_of_times_G==""):
            no_of_times_G += (str(G)+"("+str(number)+" times"+")")
            cost_G += (str(G)+"("+str(cost)+" Rs"+")")
        else:
            no_of_times_G += (", "+str(G)+"("+str(number)+" times"+")")
            cost_G += (", "+str(G)+"("+str(cost)+" Rs"+")")
        total_cost_G += cost


no_of_times_S = ""
cost_S = ""
total_cost_S = 0
for S in svs:
    number = 0
    cost = 0
    for R in rvs:
        for T in time:
            number =  number + Xrst[T,S,R].x
            cost = cost + Krst[T-1][S-1][R-1]*Nrst[T,S,R].x
    cost = round(cost)
    if(number!=0):
        if(no_of_times_S==""):
            no_of_times_S += (str(S)+"("+str(number)+" times"+")")
            cost_S += (str(S)+"("+str(cost)+" Rs"+")")
        else:
            no_of_times_S += (", "+str(S)+"("+str(number)+" times"+")")
            cost_S += (", "+str(S)+"("+str(cost)+" Rs"+")")
        total_cost_S += cost


no_of_times_R = ""
cost_R = ""
total_cost_R = 0
for R in rvs:
    number = 0
    cost = 0
    for D in dvs:
        for T in time:
            number =  number + Xdrt[T,R,D].x
            cost = cost + Kdrt[T-1][R-1][D-1]*Ndrt[T,R,D].x
    cost = round(cost)
    if(number!=0):
        if(no_of_times_R==""):
            no_of_times_R += (str(R)+"("+str(number)+" times"+")")
            cost_R += (str(R)+"("+str(cost)+" Rs"+")")
        else:
            no_of_times_R += (", "+str(R)+"("+str(number)+" times"+")")
            cost_R += (", "+str(R)+"("+str(cost)+" Rs"+")")
        total_cost_R += cost

no_of_times_D = ""
cost_D = ""
total_cost_D = 0
for D in dvs:
    number = 0
    cost = 0
    for I in clinics:
        for T in time:
            number =  number + Xidt[T,D,I].x
            cost = cost + Kidt[T-1][D-1][I-1]*Nidt[T,D,I].x
    cost = round(cost)
    if(number!=0):
        if(no_of_times_D==""):
            no_of_times_D += (str(D)+"("+str(number)+" times"+")")
            cost_D += (str(D)+"("+str(cost)+" Rs"+")")
        else:
            no_of_times_D += (", "+str(D)+"("+str(number)+" times"+")")
            cost_D += (", "+str(D)+"("+str(cost)+" Rs"+")")
        total_cost_D += cost


transport_summary = {
    "From":["M->G","G->S","S->R","R->D","D->I"],
    "Number of times transport occurs over the entire planning horizon":[no_of_times_M,no_of_times_G,no_of_times_S,no_of_times_R,no_of_times_D],
    "Cost Incurred":[cost_M,cost_G,cost_S,cost_R,cost_D],
    "Total Cost":[total_cost_M,total_cost_G,total_cost_S,total_cost_R,total_cost_D]
}


################ ordering part ##############

average_quantity_M = ""
cost_M = ""
total_cost_M = 0
for M in manufacturers:
    number = 0
    cost = 0
    quantity = 0
    for G in gmsd:
        for T in time:
            number =  number + Xgmt[T,M,G].x
            cost = cost + Cgmt[T-1][M-1][G-1]*Xgmt[T,M,G].x
            quantity = quantity + Qgmt[T,M,G].x
    cost = round(cost)
    if(number!=0):
        average_quantity = quantity/number
        if(average_quantity_M==""):
            average_quantity_M += (str(M)+"("+str(round(average_quantity))+" units"+")")
            cost_M += (str(M)+"("+str(cost)+" Rs"+")")
        else:
            average_quantity_M += (", "+str(M)+"("+str(round(average_quantity))+" units"+")")
            cost_M += (", "+str(M)+"("+str(cost)+" Rs"+")")
        total_cost_M += cost


average_quantity_G = ""
cost_G = ""
total_cost_G = 0
for G in gmsd:
    number = 0
    cost = 0
    quantity = 0
    for S in svs:
        for T in time:
            number =  number + Xsgt[T,G,S].x
            cost = cost + Csgt[T-1][G-1][S-1]*Xsgt[T,G,S].x
            quantity = quantity + Qsgt[T,G,S].x
    cost = round(cost)
    if(number!=0):
        average_quantity = quantity/number
        if(average_quantity_G==""):
            average_quantity_G += (str(G)+"("+str(round(average_quantity))+" units"+")")
            cost_G += (str(G)+"("+str(cost)+" Rs"+")")
        else:
            average_quantity_G += (", "+str(G)+"("+str(round(average_quantity))+" units"+")")
            cost_G += (", "+str(G)+"("+str(cost)+" Rs"+")")
        total_cost_G += cost


average_quantity_S = ""
cost_S = ""
total_cost_S = 0
for S in svs:
    number = 0
    cost = 0
    quantity = 0
    for R in rvs:
        for T in time:
            number =  number + Xrst[T,S,R].x
            cost = cost + Crst[T-1][S-1][R-1]*Xrst[T,S,R].x
            quantity = quantity + Qrst[T,S,R].x
    cost = round(cost)
    if(number!=0):
        average_quantity = quantity/number
        if(average_quantity_S==""):
            average_quantity_S += (str(S)+"("+str(round(average_quantity))+" units"+")")
            cost_S += (str(S)+"("+str(cost)+" Rs"+")")
        else:
            average_quantity_S += (", "+str(S)+"("+str(round(average_quantity))+" units"+")")
            cost_S += (", "+str(S)+"("+str(cost)+" Rs"+")")
        total_cost_S += cost


average_quantity_R = ""
cost_R = ""
total_cost_R = 0
for R in rvs:
    number = 0
    cost = 0
    quantity = 0
    for D in dvs:
        for T in time:
            number =  number + Xdrt[T,R,D].x
            cost = cost + Cdrt[T-1][R-1][D-1]*Xdrt[T,R,D].x
            quantity = quantity + Qdrt[T,R,D].x
    cost = round(cost)
    if(number!=0):
        average_quantity = quantity/number
        if(average_quantity_R==""):
            average_quantity_R += (str(R)+"("+str(round(average_quantity))+" units"+")")
            cost_R += (str(R)+"("+str(cost)+" Rs"+")")
        else:
            average_quantity_R += (", "+str(R)+"("+str(round((average_quantity)))+" units"+")")
            cost_R += (", "+str(R)+"("+str(cost)+" Rs"+")")
        total_cost_R += cost

average_quantity_D = ""
cost_D = ""
total_cost_D = 0
for D in dvs:
    number = 0
    cost = 0
    quantity = 0
    for I in clinics:
        for T in time:
            number =  number + Xidt[T,D,I].x
            cost = cost + Cidt[T-1][D-1][I-1]*Xidt[T,D,I].x
            quantity = quantity + Qidt[T,D,I].x
    cost = round(cost)
    if(number!=0):
        average_quantity = quantity/number
        if(average_quantity_D==""):
            average_quantity_D += (str(D)+"("+str(round(average_quantity))+" units"+")")
            cost_D += (str(D)+"("+str(cost)+" Rs"+")")
        else:
            average_quantity_D += (", "+str(D)+"("+str(round(average_quantity))+" units"+")")
            cost_D += (", "+str(D)+"("+str(cost)+" Rs"+")")
        total_cost_D += cost


ordering_summary = {
    "From":["M->G","G->S","S->R","R->D","D->I"],
    "Average quantities ordered over the entire planning horizon":[average_quantity_M,average_quantity_G,average_quantity_S,average_quantity_R,average_quantity_D],
    "Cost Incurred":[cost_M,cost_G,cost_S,cost_R,cost_D],
    "Total Cost":[total_cost_M,total_cost_G,total_cost_S,total_cost_R,total_cost_D]
}

################ inventory part #############

no_of_times_G = ""
avg_inv_G = ""
cost_G = ""
total_cost_G = 0
for G in gmsd:
    number = 0
    cost = 0
    total_inventory_G = 0
    for T in time:
        if(Igt[T,G].x>0):
            number += 1
            total_inventory_G += Igt[T,G].x
            cost += hgt[T-1][G-1]*Igt[T,G].x
    cost = round(cost)
    if(number!=0):
        if(no_of_times_G==""):
            no_of_times_G += (str(G)+"("+str(number)+" times"+")")
            cost_G += (str(G)+"("+str(cost)+" Rs"+")")
            avg_inv_G += (str(G)+"("+str(total_inventory_G/number)+" units"+")")

        else:
            no_of_times_G += (", "+str(G)+"("+str(number)+" times"+")")
            cost_G += (", "+str(G)+"("+str(cost)+" Rs"+")")
            if(number!=0):
                avg_inv_G += (", "+str(G)+"("+str(total_inventory_G/number)+" units"+")")
            else:
                avg_inv_G += (", "+str(G)+"("+str(0)+" units"+")")
    total_cost_G += cost
        

no_of_times_S = ""
avg_inv_S = ""
cost_S = ""
total_cost_S = 0
for S in svs:
    number = 0
    cost = 0
    total_inventory_S = 0
    for T in time:
        if(Ist[T,S].x>0):
            number += 1
            total_inventory_S += Ist[T,S].x
            cost += hst[T-1][S-1]*Ist[T,S].x
    cost = round(cost)
    if(number!=0):
        if(no_of_times_S==""):
            no_of_times_S += (str(S)+"("+str(number)+" times"+")")
            cost_S += (str(S)+"("+str(cost)+" Rs"+")")
            avg_inv_S += (str(S)+"("+str(total_inventory_S/number)+" units"+")")
    
        else:
            no_of_times_S += (", "+str(S)+"("+str(number)+" times"+")")
            cost_S += (", "+str(S)+"("+str(cost)+" Rs"+")")
            if(number!=0):
                avg_inv_S += (", "+str(S)+"("+str(total_inventory_S/number)+" units"+")") 
            else: 
                avg_inv_S += (", "+str(S)+"("+str(0)+" units"+")") 
    total_cost_S += cost




no_of_times_D = ""
avg_inv_D = ""
cost_D = ""
total_cost_D = 0
for D in dvs:
    number = 0
    cost = 0
    total_inventory_D = 0
    for T in time:
        if(Idt[T,D].x>0):
            number += 1
            total_inventory_D += Idt[T,D].x
            cost += hdt[T-1][D-1]*Idt[T,D].x
    cost = round(cost)
    if(number!=0):
        if(no_of_times_D==""):
                no_of_times_D += (str(D)+"("+str(number)+" times"+")")
                cost_D += (str(D)+"("+str(cost)+" Rs"+")")
                if(number!=0):
                    avg_inv_D += (str(D)+"("+str(total_inventory_D/number)+" units"+")")
                else:
                    avg_inv_D += (str(D)+"("+str(0)+" units"+")")
        else:
            no_of_times_D += (", "+str(D)+"("+str(number)+" times"+")")
            cost_D += (", "+str(D)+"("+str(cost)+" Rs"+")")
            if(number!=0):
                avg_inv_D += (", "+str(D)+"("+str(total_inventory_D/number)+" units"+")")  
            else:
                avg_inv_D += (", "+str(D)+"("+str(0)+" units"+")")
    total_cost_D += cost


I_s = 1
total_cost_I = 0
no_of_times_I = ""
D = 1
cost_I = ""
avg_inv_I = ""
for I_b in clinic_breakpoints:
    number_average_district = 0
    avg_inv_avg_district = 0
    cost_avg_district = 0
    for I in range(I_s,I_b+1):
        number = 0
        cost = 0
        total_inventory_I = 0
        for T in time:
            if(Iit[T,I].x>0):
                number += 1
                total_inventory_I += Iit[T,I].x
                cost += hit[T-1][I-1]*Iit[T,I].x
        number_average_district += number
        if(number!=0):
            avg_inv_avg_district += total_inventory_I/number
        else:
            avg_inv_avg_district += 0
        cost_avg_district += cost
        total_cost_I += cost

    number_average_district /= (I_b-I_s+1)
    number_average_district = round(number_average_district)

    avg_inv_avg_district /= (I_b-I_s+1)
    avg_inv_avg_district = round(avg_inv_avg_district)

    cost_avg_district /= (I_b-I_s+1)

    if(no_of_times_I==""):
            no_of_times_I += (str(D)+"("+str(number_average_district)+" times"+")")
            cost_I += (str(D)+"("+str(round(cost_avg_district))+" Rs"+")")
            avg_inv_I += (str(D)+"("+str(round(avg_inv_avg_district))+" units"+")")
    else:
        no_of_times_I += (", "+str(D)+"("+str(number_average_district)+" times"+")")
        cost_I += (", "+str(D)+"("+str(round(cost_avg_district))+" Rs"+")")
        avg_inv_I += (", "+str(D)+"("+str(round(avg_inv_avg_district))+" units"+")") 

    D = D+1
    I_s = I_b+1

inventory_summary = {
    "CCP": ["G","S","R","D","I"],
    "Number of times Inventory is non zero":[no_of_times_G,no_of_times_S,no_of_times_R,no_of_times_D,no_of_times_I],
    "Average Inventory carried":[avg_inv_G,avg_inv_S,avg_inv_R,avg_inv_D,avg_inv_I],
    "Cost Incurred":[cost_G,cost_S,cost_R,cost_D,cost_I],
    "Total Cost":[total_cost_G,total_cost_S,total_cost_R,total_cost_D,total_cost_I]
}

########################### Shortages summary ##############################


I_s = 1
num1 = 0
costs_list = []
average_costs_list = []
number_of_clinics = []
shortages_list = []
count = 0
clinic_num = []
total_shortage = []
for num in clinic_breakpoints:
    cost = [0 for J in range(j)]
    fraction_of_shortages = [0 for J in range(j)]
    num_clinics = num - num1
    count += 1
    total_demand = [0 for J in range(j)]
    for x in range(num_clinics):
        I = I_s + x
        for J in customers:
            for T in time:
                total_demand[J-1] += dijt[T-1][J-1][I-1]
                if(Sijt[T,J,I].x!=0):
                    cost[J-1] += Pjt[T-1][J-1]*Sijt[T,J,I].x
                    fraction_of_shortages[J-1]+=Sijt[T,J,I].x
    total_shortages = sum(fraction_of_shortages)
    for J in customers:
        fraction_of_shortages[J-1] = fraction_of_shortages[J-1]*100/total_demand[J-1]    
        
    avg_cost = [cost[J-1]/num_clinics for J in customers]
    I_s += num_clinics
    num1 = num
    shortage_string = ""
    cost_string = ""
    avg_cost_string = ""
    for J in customers:
        cost_string += str(J)+"("+str(cost[J-1])+"), "
        avg_cost_string += str(J)+"("+str(avg_cost[J-1])+"), "
        shortage_string += str(J)+"("+str(fraction_of_shortages[J-1])+"), "

    costs_list.append(cost_string)
    average_costs_list.append(avg_cost_string)
    number_of_clinics.append(num_clinics)
    shortages_list.append(shortage_string)
    clinic_num.append(count)
    total_shortage.append(total_shortages)

shortage_summary = {
    "District Number": clinic_num,
    "Number of clinics": number_of_clinics,
    "Total shortages": total_shortage,
    "Total shortage cost Incurred": costs_list,
    "Percentage(group-wise) of shortages": shortages_list,
    "Average shortage cost incurred per clinic":average_costs_list
}

################################## Nurses part ####################################
I_s = 1
num1 = 0
avg_number_list = []
avg_hiring_list = []
avg_firing_list = []
count = 0
clinic_num = []
for num in clinic_breakpoints:
    num_clinics = num - num1
    count += 1
    n_nurses = 0
    n_hiring = 0
    n_firing = 0
    for x in range(num_clinics):
        I = I_s + x
        for T in time:
            n_nurses += N_nurses_it[T,I].x
            n_hiring += H_nurses_it[T,I].x
            n_firing += F_nurses_it[T,I].x
    
    I_s += num_clinics
    num1 = num
    avg_number_list.append(n_nurses/(num_clinics*t))
    avg_hiring_list.append(n_hiring/(num_clinics*t))
    avg_firing_list.append(n_firing/(num_clinics*t))
    clinic_num.append(count)


nurses_summary = {
"District Number": clinic_num,
"Avg number of nurses per week": avg_number_list,
"Avg number of hired nurses per week": avg_hiring_list,
"Avg number of fired nurses per week": avg_firing_list
}

transport_df = pd.DataFrame.from_dict(transport_summary)
inventory_df = pd.DataFrame.from_dict(inventory_summary)
ordering_df = pd.DataFrame.from_dict(ordering_summary)
shortage_df = pd.DataFrame.from_dict(shortage_summary)
nurses_df = pd.DataFrame.from_dict(nurses_summary)

###########################################Compiled Results##################################################
writer = pd.ExcelWriter('with_districts-'+str(d)+' fraction-'+str(fraction_storage)+' manuf-'+ str(Bmt[0][0]) +'-without-x-inventory-ini-'+str(Ii_0)+"---"+str(Lm)+"-"+str(Lg)+"-"+str(Ls)+"-"+str(Lr)+"-"+str(Ld)+"-"+str(Li)+'shortage-'+str(SC)+'.xlsx',engine='xlsxwriter')   
workbook=writer.book
worksheet=workbook.add_worksheet('Compiled')
writer.sheets['Compiled'] = worksheet
worksheet.write(0,0,"Transport part")
transport_df.to_excel(writer,sheet_name='Compiled',startrow=2 , startcol=0)

x = 4 + len(transport_df.index)
worksheet.write(x,0,"Inventory part")
x += 2
inventory_df.to_excel(writer,sheet_name='Compiled',startrow=x , startcol=0)

x += 4 + len(inventory_df.index)
worksheet.write(x,0,"Ordering part")
x += 2
ordering_df.to_excel(writer,sheet_name='Compiled',startrow=x , startcol=0)

x += 4 + len(ordering_df.index)
worksheet.write(x,0,"Shortage part")
x += 2
shortage_df.to_excel(writer,sheet_name='Compiled',startrow=x , startcol=0)

x += 4 + len(shortage_df.index)
worksheet.write(x,0,"Nurses part")
x += 2
nurses_df.to_excel(writer,sheet_name='Compiled',startrow=x , startcol=0)
workbook.close()

################################################# The End ######################################################### 

In [13]:
################################### Clinic-DVS relation #####################################
dvss = []      
for I in clinics:
    temp = []
    for T in time:    
        for D in dvs:
            if(Xidt[T,D,I].x==1 and D not in top_dvs_arr[I-1]):
                    print(I)
                    print(D)
                    print(T)
                    print("")
            if(Xidt[T,D,I].x==1 and D not in temp):
                temp.append(D)
                
    dvss.append(temp)
clinic_breakpoints = [10,16,28,40,59,76,92,103,123,151,178,194,205,213,226,249,256,266,274,290,312,322,340,361,376,413,432,451,462,483,504,511,517,535,555,568,585,606]   #CLinic breakpoints for each districts
arr = []
clinic_no = 1 
dd = 1
for c in clinic_breakpoints:
    while(clinic_no<=c):
        arr.append(dd)
        clinic_no = clinic_no+1
    dd = dd+1 
arr = arr[:i]
dict_dvs = {
    "Clinincs": range(1,i+1),
    "DVSs":dvss,
    "original":arr,
    "Dist from 1": [Did[0][I-1] for I in clinics],
    "Dist from 2": [Did[1][I-1] for I in clinics],
    "Dist from 3": [Did[2][I-1] for I in clinics],
    "Dist from 4": [Did[3][I-1] for I in clinics],
    "Dist from 5": [Did[4][I-1] for I in clinics],
    "Dist from 6": [Did[5][I-1] for I in clinics],
    "Dist from 7": [Did[6][I-1] for I in clinics],
    "Dist from 8": [Did[7][I-1] for I in clinics],
    "Dist from 9": [Did[8][I-1] for I in clinics],
    "Dist from 10": [Did[9][I-1] for I in clinics],
    "Dist from 11": [Did[10][I-1] for I in clinics],
    "Dist from 12": [Did[11][I-1] for I in clinics],
    "Dist from 13": [Did[12][I-1] for I in clinics],
    "Dist from 14": [Did[13][I-1] for I in clinics],
    "Dist from 15": [Did[14][I-1] for I in clinics],
    "Dist from 16": [Did[15][I-1] for I in clinics],
    "Dist from 17": [Did[16][I-1] for I in clinics],
    "Dist from 18": [Did[17][I-1] for I in clinics],
    "Dist from 19": [Did[18][I-1] for I in clinics],
    "Dist from 20": [Did[19][I-1] for I in clinics],
    "Dist from 21": [Did[20][I-1] for I in clinics],
    "Dist from 22": [Did[21][I-1] for I in clinics],
    "Dist from 23": [Did[22][I-1] for I in clinics],
    "Dist from 24": [Did[23][I-1] for I in clinics],
    "Dist from 25": [Did[24][I-1] for I in clinics],
    "Dist from 26": [Did[25][I-1] for I in clinics],
    "Dist from 27": [Did[26][I-1] for I in clinics],
    "Dist from 28": [Did[27][I-1] for I in clinics],
    "Dist from 29": [Did[28][I-1] for I in clinics],
    "Dist from 30": [Did[29][I-1] for I in clinics],
    "Dist from 31": [Did[30][I-1] for I in clinics],
    "Dist from 32": [Did[31][I-1] for I in clinics],
    "Dist from 33": [Did[32][I-1] for I in clinics],
    "Dist from 34": [Did[33][I-1] for I in clinics],
    "Dist from 35": [Did[34][I-1] for I in clinics],
    "Dist from 36": [Did[35][I-1] for I in clinics],
    "Dist from 37": [Did[36][I-1] for I in clinics],
    "Dist from 38": [Did[37][I-1] for I in clinics],
}
dvs_df = pd.DataFrame.from_dict(dict_dvs)
# print(dvs_df)

          

# print(dvss)
# print(rvs_df)
writer = pd.ExcelWriter('no-pre-temp'+str(d)+'.xlsx',engine='xlsxwriter')   
workbook=writer.book
worksheet=workbook.add_worksheet('Compiled')
writer.sheets['Compiled'] = worksheet
worksheet.write(0,0,"DVS Selected")
dvs_df.to_excel(writer,sheet_name='Compiled',startrow=2 , startcol=0)

workbook.close()

6
20
8

7
18
5

7
18
11

17
25
3

28
35
5

28
25
11

29
16
1

30
16
1

31
16
1

31
20
2

32
16
1

33
16
1

36
16
1

37
16
1

37
36
5

37
25
11

38
16
1

38
18
5

39
16
1

40
16
1

40
25
3

45
18
5

46
1
1

55
36
5

56
36
5

62
18
5

64
24
7

69
18
5

75
17
1

78
35
5

80
35
5

81
6
5

82
35
5

88
37
3

93
3
1

96
3
1

97
3
1

101
35
5

103
3
1

103
35
5

108
36
5

110
29
1

110
11
9

113
29
1

113
36
5

113
2
9

116
29
1

118
36
5

119
36
5

120
29
1

120
2
9

121
36
5

125
35
5

125
2
11

126
35
5

127
35
5

127
2
11

129
35
5

134
35
5

134
2
11

135
35
5

136
18
5

137
35
5

137
2
11

139
35
5

139
2
11

140
35
5

141
35
1

143
35
5

144
35
5

145
35
5

145
2
11

146
35
5

146
2
11

148
18
5

149
35
1

149
13
2

149
35
5

150
18
5

152
3
1

154
3
1

155
3
1

158
3
1

165
3
1

166
26
1

166
18
5

168
3
1

171
3
1

173
18
5

174
3
1

175
3
1

175
23
2

178
3
1

197
18
5

198
11
9

199
11
9

202
26
1

204
17
1

207
3
1

209
3
1

211
3
1

211
36
5

212
3
1

213
3
1

229
9
2

233
33
3

2

In [ ]:
############################# Saving solution ##########################################
name = []
value = []
for T in time:
    for G in gmsd:
        name.append(Igt[T,G].varName)
        value.append(Igt[T,G].x)
    for S in svs:
        name.append(Ist[T,S].varName)
        value.append(Ist[T,S].x)
    for D in svs:
        name.append(Idt[T,D].varName)
        value.append(Idt[T,D].x)
    for I in clinics:
        name.append(Iit[T,I].varName)
        value.append(Iit[T,I].x)
inventory_dict = {
    "Name" : name,
    "Value" : value
}
inv_df = pd.DataFrame.from_dict(inventory_dict)
inv_df.to_csv('inv.csv')

name = []
value = []
for T in time:
    for M in manufacturers:
        for G in gmsd:
            name.append(Qgmt[T,M,G].varName)
            value.append(Qgmt[T,M,G].x)
    for G in gmsd:
        for S in svs:
            name.append(Qsgt[T,G,S].varName)
            value.append(Qsgt[T,G,S].x)
    for S in svs:
        for D in dvs:
            name.append(Qdst[T,S,D].varName)
            value.append(Qdst[T,S,D].x)
    for D in dvs:
        for I in clinics:
            name.append(Qidt[T,D,I].varName)
            value.append(Qidt[T,D,I].x)
quantity_dict = {
    "Name" : name,
    "Value" : value
}
qty_df = pd.DataFrame.from_dict(quantity_dict)
qty_df.to_csv('qty.csv')

name = []
value = []
for T in time:
    for I in clinics:
        for J in customers:
            name.append(Wijt[T,J,I].varName)
            value.append(Wijt[T,J,I].x)
            name.append(Sijt[T,J,I].varName)
            value.append(Sijt[T,J,I].x)
consumption_dict = {
    "Name" : name,
    "Value" : value
}
con_df = pd.DataFrame.from_dict(consumption_dict)
con_df.to_csv('cons.csv')

name = []
value = []
for T in time:
    for M in manufacturers:
        for G in gmsd:
            name.append(Xgmt[T,M,G].varName)
            value.append(Xgmt[T,M,G].x)
    for G in gmsd:
        for S in svs:
            name.append(Xsgt[T,G,S].varName)
            value.append(Xsgt[T,G,S].x)
    for S in svs:
        for D in dvs:
            name.append(Xdst[T,S,D].varName)
            value.append(Xdst[T,S,D].x)
    for D in dvs:
        for I in clinics:
            name.append(Xidt[T,D,I].varName)
            value.append(Xidt[T,D,I].x)
assignment_dict = {
    "Name" : name,
    "Value" : value
}
agn_df = pd.DataFrame.from_dict(assignment_dict)
agn_df.to_csv('agn.csv')

name = []
value = []
for T in time:
    for M in manufacturers:
        for G in gmsd:
            name.append(Ngmt[T,M,G].varName)
            value.append(Ngmt[T,M,G].x)
    for G in gmsd:
        for S in svs:
            name.append(Nsgt[T,G,S].varName)
            value.append(Nsgt[T,G,S].x)
    for S in svs:
        for D in dvs:
            name.append(Ndst[T,S,D].varName)
            value.append(Ndst[T,S,D].x)
    for D in dvs:
        for I in clinics:
            name.append(Nidt[T,D,I].varName)
            value.append(Nidt[T,D,I].x)
trucks_dict = {
    "Name" : name,
    "Value" : value
}
trk_df = pd.DataFrame.from_dict(trucks_dict)
trk_df.to_csv('trk.csv')

name = []
value = []
for T in time:
    for I in clinics:
        name.append(N_nurses_it[T,I].varName)
        value.append(N_nurses_it[T,I].x)
        name.append(H_nurses_it[T,I].varName)
        value.append(H_nurses_it[T,I].x)
        name.append(F_nurses_it[T,I].varName)
        value.append(F_nurses_it[T,I].x)
nurses_dict = {
    "Name" : name,
    "Value" : value
}
nur_df = pd.DataFrame.from_dict(nurses_dict)
nur_df.to_csv('nur.csv')